In [14]:
from __future__ import absolute_import, division, print_function

import pandas as pd
import numpy as np
import sklearn as sk
import os
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import h5py as h5py
from time import time
# %load_ext autoreload
# %autoreload 2

In [ ]:
data = np.load("embedded_data.npz")

In [3]:
data.files

['x_train_e', 'y_train_e', 'x_valid_e', 'y_valid_e', 'x_test_e', 'y_test_e']

In [4]:
max_episode = 10

def get_padded_matrix(x_, y_):
    episode_matrix = np.zeros(shape=[x_.shape[0], max_episode, x_.shape[1]])
    episode = []
    for idx in range(len(x_)):
        # within on episode
        episode.append(x_[idx])
        for event_idx in range(min(len(episode), max_episode)):
            episode_matrix[idx][-(event_idx%max_episode)-1] = episode[-event_idx - 1]
        if np.equal(x_[idx][:-1],y_[idx]).all():
            # episode changed
            episode = []
    return episode_matrix

In [5]:
train_matrix = get_padded_matrix( data['x_train_e'],  data['y_train_e'])

In [6]:
valid_matrix = get_padded_matrix( data['x_valid_e'],  data['y_valid_e'])

In [7]:
train_matrix.shape

(153582, 10, 11)

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, input_shape=(train_matrix.shape[1], train_matrix.shape[2])),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(train_matrix.shape[2]-1)
])

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                12400     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               6528      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 20,218
Trainable params: 20,218
Non-trainable params: 0
_________________________________________________________________


In [11]:
optimizer = keras.optimizers.Adam()
model.compile(optimizer='adam', loss='mean_squared_error')

In [12]:
try:
    model.load_weights("model_embed_weights.h5py")
    print("Loaded saved model :)")
except OSError:
    print("No saved model. Training from scratch.")

No saved model. Training from scratch.


In [15]:
tensorboard = keras.callbacks.TensorBoard(log_dir="logs/lstm_embed{}".format(time()))
model.fit(train_matrix, data['y_train_e'], validation_data=(valid_matrix, data['y_valid_e']), epochs=10, callbacks=[tensorboard])


Train on 153582 samples, validate on 19527 samples
Epoch 1/10
153582/153582 [==============================] - 65s 426us/step - loss: 6.6077 - val_loss: 3.0717
Epoch 2/10
153582/153582 [==============================] - 60s 392us/step - loss: 5.1310 - val_loss: 2.9218
Epoch 3/10
153582/153582 [==============================] - 62s 405us/step - loss: 5.0857 - val_loss: 2.8995
Epoch 4/10
153582/153582 [==============================] - 65s 422us/step - loss: 5.0597 - val_loss: 2.9260
Epoch 5/10
153582/153582 [==============================] - 62s 405us/step - loss: 5.0393 - val_loss: 2.9320
Epoch 6/10
153582/153582 [==============================] - 64s 416us/step - loss: 5.0362 - val_loss: 2.8794
Epoch 7/10
153582/153582 [==============================] - 69s 451us/step - loss: 5.0156 - val_loss: 2.8428
Epoch 8/10
153582/153582 [==============================] - 50s 324us/step - loss: 5.0118 - val_loss: 2.9129
Epoch 9/10
153582/153582 [==============================] - 51s 333us/step - 

In [16]:
model.save_weights("model_embed_weights.h5py")

In [22]:
len(columns)

46

In [23]:
columns.index("HeartRate")

11